In [22]:
import os
import sys
import torch

import plotly
import numpy as np
import plotly.graph_objs as go
from sklearn.decomposition import PCA

sys.path.append('..', '..')
from src.utils import int_to_pitch 

src_path = os.path.join('..')

In [23]:
model_path = os.path.join(
    src_path,
    'mlruns',
#     '2/528338104be34c8b90cf934bc1cc75ea', 
#     '2/603b3504919e46b3980a9ddfcc110fcf',
#     '2/9d6f067d0c804a99af08c6e2a67bd688',
#     '5/8fcc0502375b4611861a8a532530f57c',
#     '2/9d6f067d0c804a99af08c6e2a67bd688', 
#     '2/e30ecc06a2854d58a731bed4821eff7b',
#     '5/0771fdebe48e4a27a7e227f90c5acea6',
#     '5/35a6d6d2fc364c7c89f121dca3d90a5a',
    '5/8d5a71e46daa42788fbe1a701e882d14',
    'artifacts',
#     '22_05_21_10_09_12_transpose_all_chord_extended_7_batchsize_64_seed_1234567890_early_stop.pt',
#     '22_05_22_00_52_58_transpose_all_chord_extended_7_batchsize_128_seed_1234567890.pt',
#     '22_05_20_13_49_37_transpose_all_chord_extended_7_batchsize_64_seed_1234567890.pt',
#     '22_05_28_20_53_27_transpose_all_chord_extended_7_batchsize_64_seed_1234567890.pt',
#     '22_05_20_13_49_37_transpose_all_chord_extended_7_batchsize_64_seed_1234567890.pt',
#     'transpose_all_chord_extended_7_batchsize_64_seed_1234567890_best_val.pt',
#     '22_05_30_14_10_32_transpose_all_chord_extended_7_batchsize_64_seed_1234567890_early_stop.pt',
#     '22_05_30_21_47_05_transpose_all_chord_extended_7_batchsize_64_seed_1234567890_early_stop.pt',
    '22_05_30_22_52_17_transpose_all_chord_extended_7_batchsize_64_seed_1234567890.pt'
)

model = torch.load(open(model_path, 'rb'))

In [24]:
def display_pca_scatterplot(embedding, user_input=None, labels=None, opacity=None, colors=None, fontsize=18, markersize=20, textposition='top left', cam=(0,0,0), dims=3, avg=False):
    emb_vectors = np.array([
        embedding(
            torch.Tensor([note]).long().cuda()).cpu().detach().numpy().squeeze()
                  for note in user_input
    ])

    pca = PCA(random_state=0).fit_transform(emb_vectors)[:,:dims]

    data = []
  
    if avg:
        xx = []
        yy = []

        for i in range(12):
            xxs = np.array([])
            yys = np.array([])
                                        
            for j in range(len(pca[:,0]) // 12):
                xxs = np.append(xxs, pca[:,0][12*j+i])
                yys = np.append(yys, pca[:,1][12*j+i]) 
            
            xx.append(xxs.mean())
            yy.append(yys.mean())
        
        xx = np.array(xx)
        xx = np.append(pca[:, 0], xx)

        yy = np.array(yy)
        yy = np.append(pca[:, 1], yy)
        
        if dims == 3:
            zz = []
            
            for i in range(12):
                zzs = np.array([])
                
                for j in range(len(pca[:,0]) // 12):
                    zzs = np.append(xxs, pca[:,2][12*j+i])
                
                zz.append(zzs.mean())
   
            zz = np.array(zz)
            zz = np.append(pca[:, 2], zz)
    else:
        xx = pca[:, 0]
        yy = pca[:, 1]
        
        if dims == 3:
            zz = pca[:, 2]
    
    marker = {
        'size': markersize,
        'opacity': opacity if dims == 2 else 0.8,
        'color': colors
    }
    
    if dims == 2:
        trace = go.Scatter(
            x = xx, 
            y = yy,  
            text = labels,
            textposition = textposition,
            textfont_size = fontsize,
            mode = 'markers+text',
            marker = marker
        )
    else:      
        trace = go.Scatter3d(
            x = xx, 
            y = yy, 
            z = zz,
            text = labels,
            textposition = textposition,
            textfont_size = fontsize,
            mode = 'markers+text',
            marker = marker
        )

    data.append(trace)

    # Configure the layout

    layout = go.Layout(
        margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
        showlegend=False,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        legend=dict(
            x=1,
            y=0.5,
            font=dict(
                family="Courier New",
                size=20,
                color="black"
            )
        ),
        font = dict(
            family = "Courier New",
            size = 12
        ),
        autosize = False,
        width = 1000,
        height = 1000,
        scene_camera=dict(
            eye=dict(x=cam[0], y=cam[1], z=cam[2])
        )
    )

    plot_figure = go.Figure(data = data, layout = layout)
    plot_figure.show()

In [42]:
user_input = list(range(48, 84)) + [128, 129]

opacity = list(np.repeat(1, 12*3)) + list(np.repeat(1, 2)) + list(np.repeat(0.4, 12))

# labels = ['C2' ,'Db2', 'D2', 'Eb2', 'E2', 'F2', 'F#2', 'G2', 'Ab2', 'A2', 'Bb2', 'B2'] +\
#          ['C3' ,'Db3', 'D3', 'Eb3', 'E3', 'F3', 'F#3', 'G3', 'Ab3', 'A3', 'Bb3', 'B3'] +\
#          ['C4' ,'Db4', 'D4', 'Eb4', 'E4', 'F4', 'F#4', 'G4', 'Ab4', 'A4', 'Bb4', 'B4'] +\
#          ['C4' ,'Db4', 'D4', 'Eb4', 'E4', 'F4', 'F#4', 'G4', 'Ab4', 'A4', 'Bb4', 'B4'] +\

labels =  list(np.repeat("", 12*3)) + \
         ['R', 'P'] + \
         ['C' ,'Db', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'Ab', 'A', 'Bb', 'B']

colors = [(i % 12)*3 for i in range(12*3)] + ['grey', 'darkgrey'] + [(i % 12)*3 for i in range(12)]

display_pca_scatterplot(model.pitch_encoder, user_input, labels, opacity, colors, 
                        fontsize=32, markersize=32, textposition='middle left', cam=(0,0,0), dims=2, avg=True)

In [35]:
user_input = list(range(48))

# labels = [f'{i}' for i in user_input]
labels = list(np.repeat("", len(user_input))) + [f'{i%12}' for i in user_input]

opacity = list(np.repeat(1, 12*4))  + list(np.repeat(0.4, 12))

colors = [i%12 for i in user_input] + [i % 12 for i in range(12)]

display_pca_scatterplot(model.offset_encoder, user_input, labels, opacity, colors, 
                        fontsize=32, markersize=32, textposition='bottom left', cam=(0.5,0,0), dims=2, avg=True)

In [20]:
durations = torch.Tensor(list(range(1, 75)))
idx = model.convert_durations_to_ids(durations).long().tolist()


labels = [f'{i}' if i <= 13 else "" for i in durations.long().tolist()]
offset = 0.8
colors = [i for i in idx] 

display_pca_scatterplot(model.duration_encoder, idx, labels, fontsize=32, textposition='top center', markersize=32, dims=2)